# The Experiment Container

This notebook explains how the database works as an experiment container.

## Required imports

In [1]:
import os
import qcodes as qc
from qcodes.dataset.sqlite.database import initialise_or_create_database_at, connect
from qcodes.dataset.data_set import new_data_set
from qcodes.dataset.experiment_container import (experiments, load_experiment,
                                                 load_last_experiment,new_experiment,
                                                 load_experiment_by_name,
                                                 load_or_create_experiment)
from qcodes.dataset.experiment_settings import (reset_default_experiment_id,
                                                get_default_experiment_id)

## The experiments inside the database

In [2]:
db_file_path = os.path.join(os.getcwd(), 'exp_container_tutorial.db')
initialise_or_create_database_at(db_file_path)

The database holds a certain number of **experiments**. They may be viewed:

In [3]:
experiments()

[]

Not surprisingly, our new database is empty.

We now introduce a term that we call the **default experiment**. In short, it is the experiment that will be used for a QCoDeS `DataSet`, if the user do not explicitly pass an experiment into objects that create that `DataSet`. In another word, that `DataSet` will be belong to the default experiment. We do not want to go into the details of `DataSet` here, and refer to the [DataSet notebook](https://qcodes.github.io/Qcodes/examples/DataSet/DataSet-class-walkthrough.html) and [Performing measurements using qcodes parameters and dataset](https://qcodes.github.io/Qcodes/examples/DataSet/Performing-measurements-using-qcodes-parameters-and-dataset.html) for what we mean from `DataSet` and how we can pass an experiment explicitly.

By default, the last experiment in the database is the default experiment. The default experiment can be changed if another experiment in the database is created or loaded. We will explore this in this notebook. 

Users should not worry about checking the default experiment in their normal workflow, but in this notebook, we show how it works to let them have an idea what the default experiment is and how it changes.  

We need a connection to our database to get which experiment is the default one:

In [4]:
conn = connect(db_file_path)

Because our database is empty now, asking for the default experiment will rasie an error asking to create an experiment. So, let's add some experiments to explore more:

In [5]:
exp_a = new_experiment('first_exp', sample_name='old_sample')
exp_b = new_experiment('second_exp', sample_name='slightly_newer_sample')
exp_c = load_or_create_experiment('third_exp', sample_name='brand_new_sample')

We recommend using the `load_or_create_experiment` function as the primary function dealing with experiments, not only because it is the most versatile function, but also because it can prevent creating duplicate experiments in one database.

In [6]:
experiments()

[first_exp#old_sample#1@/home/runner/work/Qcodes/Qcodes/docs/examples/DataSet/exp_container_tutorial.db
 ------------------------------------------------------------------------------------------------------,
 second_exp#slightly_newer_sample#2@/home/runner/work/Qcodes/Qcodes/docs/examples/DataSet/exp_container_tutorial.db
 ------------------------------------------------------------------------------------------------------------------,
 third_exp#brand_new_sample#3@/home/runner/work/Qcodes/Qcodes/docs/examples/DataSet/exp_container_tutorial.db
 ------------------------------------------------------------------------------------------------------------]

We notice that each experiment is labelled by an integer number, which is the `exp_id`. This ID can be used when looking up properties of each experiment.

Let's check to see which experiment is the default now:

In [7]:
get_default_experiment_id(conn)

3

The latest created or loaded experiment in the database becomes the default experiment, and the function returns the `exp_id` of that experiment, which in this case it is `exp_c` with `exp_id` of 3.

Let us add some `DataSet` to our experiments. For the sake of clarity, we don't add any data to the `DataSet` here, and refer to the above-mentioned notebooks for the details. Note that the `new_data_set` function is used here ONLY for the sake of exercise and should NOT be used in the actual experiment.

In [8]:
new_data_set('run_a')

run_a #1@/home/runner/work/Qcodes/Qcodes/docs/examples/DataSet/exp_container_tutorial.db
----------------------------------------------------------------------------------------

Since the default experiment is exp_c (`exp_id`=3), the above `DataSet` belongs to this experiment.

In [9]:
exp_c

third_exp#brand_new_sample#3@/home/runner/work/Qcodes/Qcodes/docs/examples/DataSet/exp_container_tutorial.db
------------------------------------------------------------------------------------------------------------
1-run_a-1-None-0

Let's load another experiment (`exp_b`). We know that the latest created/ loaded experiment should be the default experiment, meaning any new `DataSet` should belong to this experiment:

In [10]:
load_or_create_experiment('second_exp', sample_name='slightly_newer_sample')

second_exp#slightly_newer_sample#2@/home/runner/work/Qcodes/Qcodes/docs/examples/DataSet/exp_container_tutorial.db
------------------------------------------------------------------------------------------------------------------

Let's confirm that actually the second experiment (`exp_id`=2) is the default now:

In [11]:
get_default_experiment_id(conn)

2

In [12]:
new_data_set('first_run_b')

first_run_b #2@/home/runner/work/Qcodes/Qcodes/docs/examples/DataSet/exp_container_tutorial.db
----------------------------------------------------------------------------------------------

In [13]:
new_data_set('second_run_b')

second_run_b #3@/home/runner/work/Qcodes/Qcodes/docs/examples/DataSet/exp_container_tutorial.db
-----------------------------------------------------------------------------------------------

Two above `DataSet`s should belong to `exp_b`:

In [14]:
exp_b

second_exp#slightly_newer_sample#2@/home/runner/work/Qcodes/Qcodes/docs/examples/DataSet/exp_container_tutorial.db
------------------------------------------------------------------------------------------------------------------
2-first_run_b-1-None-0
3-second_run_b-2-None-0

We can also explicitly use `exp_id` in creating `DataSet`s, so let's add a `DataSet` to the first experiment:

In [15]:
new_data_set('first_run', exp_id=1)

first_run #4@/home/runner/work/Qcodes/Qcodes/docs/examples/DataSet/exp_container_tutorial.db
--------------------------------------------------------------------------------------------

In [16]:
exp_a

first_exp#old_sample#1@/home/runner/work/Qcodes/Qcodes/docs/examples/DataSet/exp_container_tutorial.db
------------------------------------------------------------------------------------------------------
4-first_run-1-None-0

The default experiment gets reset upon initialization of a database. Let's check this by initializing our database again (note that our database is not empty anymore):

In [17]:
initialise_or_create_database_at(db_file_path)

The default experiment was `exp_id`=2. As we initialized our database again, the default experiment has been reset, meaning the last experiment in the database should be the default one now (we know the last experiment in the database is `exp_id`=3). Let's check this:

In [18]:
get_default_experiment_id(conn)

3

Users may not need to use the reset function explicitly, but in the case they want to use it, here we show how to do that:

First, we load an experiment other than the last experiment and check the default experiment is the just loaded experiment:

In [19]:
load_experiment(1)

first_exp#old_sample#1@/home/runner/work/Qcodes/Qcodes/docs/examples/DataSet/exp_container_tutorial.db
------------------------------------------------------------------------------------------------------
4-first_run-1-None-0

In [20]:
get_default_experiment_id(conn)

1

Now, we reset the default experiment and expect to see the last experiment (`exp_id`=3) is the default one:

In [21]:
reset_default_experiment_id() # the explicit database connection can be used as an optional argument 

In [22]:
get_default_experiment_id(conn)

3

Let's make sure it is truly the default experiment by creating a new `DataSet`:

In [23]:
new_data_set('default_run')

default_run #5@/home/runner/work/Qcodes/Qcodes/docs/examples/DataSet/exp_container_tutorial.db
----------------------------------------------------------------------------------------------

This `DataSet` should belong to `exp_c`:

In [24]:
exp_c

third_exp#brand_new_sample#3@/home/runner/work/Qcodes/Qcodes/docs/examples/DataSet/exp_container_tutorial.db
------------------------------------------------------------------------------------------------------------
1-run_a-1-None-0
5-default_run-2-None-0

There are a few other useful functions to load experiments:

In [25]:
load_experiment_by_name('second_exp', sample='slightly_newer_sample')  # loads using name and sample

second_exp#slightly_newer_sample#2@/home/runner/work/Qcodes/Qcodes/docs/examples/DataSet/exp_container_tutorial.db
------------------------------------------------------------------------------------------------------------------
2-first_run_b-1-None-0
3-second_run_b-2-None-0

In [26]:
load_last_experiment() # loads the last experiment in the database

third_exp#brand_new_sample#3@/home/runner/work/Qcodes/Qcodes/docs/examples/DataSet/exp_container_tutorial.db
------------------------------------------------------------------------------------------------------------
1-run_a-1-None-0
5-default_run-2-None-0